In [180]:
import os
import sys 
sys.path.insert(0, '..')

In [182]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from helper import _process_pred_labels, save_to_pickle
from evaluate import get_one_opt_treshold
from sklearn.metrics import f1_score
from utils import squeeze_pred_proba
from config import HANDCRAFTED_DATA_PATH

In [183]:
RANDOM_STATE = 789
N_SPLITS = 5

In [184]:
np.random.seed(RANDOM_STATE)

In [185]:
model_names = ['keras', 'catboost_labse_en_ru', 'sklearn', 'catboost', 'catboost_rubert_tiny', 'catboost_sbert_large_nlu_ru']
oof_pred_proba = pd.concat([
    (pd.read_csv(f'../oof_predictions/{model_name}/{model_name}_pred_proba.csv', index_col='review_id')
       .rename(columns={c: f'{model_name}_{c}' for c in map(str, range(9))}))
    for model_name in model_names], axis=1)
test_pred_proba = pd.concat([
    (pd.read_csv(f'../test_predictions/{model_name}/{model_name}_pred_proba.csv', index_col='review_id')
       .rename(columns={c: f'{model_name}_{c}' for c in map(str, range(9))}))
    for model_name in model_names], axis=1)
target = pd.read_pickle('../data/prepared/target.pkl')

In [186]:
oof_pred_proba  = oof_pred_proba.join(pd.read_pickle(os.path.join(HANDCRAFTED_DATA_PATH, 'train.pkl')))
test_pred_proba  = test_pred_proba.join(pd.read_pickle(os.path.join(HANDCRAFTED_DATA_PATH, 'test.pkl')))

In [192]:
cv = MultilabelStratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
base_model = LGBMClassifier()
multiout_model = MultiOutputClassifier(estimator=base_model, n_jobs=-1)


params = {} # 'estimator__C': np.random.uniform(0.1, 50, 50)
gscv = GridSearchCV(
    estimator=multiout_model, 
    param_grid=params,
    scoring='f1_samples',
    cv=cv,
    verbose=5,
    n_jobs=-1)

In [193]:
gscv.fit(X=oof_pred_proba, y=target)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 5/5] END ...estimator__C=21.202293518715507;, score=0.828 total time= 4.9min
[CV 1/5] END ...estimator__C=21.570840306762204;, score=0.830 total time= 6.5min
[CV 3/5] END ...estimator__C=21.202293518715507;, score=0.836 total time= 6.7min
[CV 2/5] END ...estimator__C=21.202293518715507;, score=0.835 total time= 6.7min
[CV 1/5] END ...estimator__C=21.202293518715507;, score=0.829 total time= 6.7min
[CV 3/5] END ...estimator__C=21.570840306762204;, score=0.836 total time= 6.8min
[CV 4/5] END ...estimator__C=21.202293518715507;, score=0.835 total time= 7.3min
[CV 2/5] END ...estimator__C=21.570840306762204;, score=0.834 total time= 8.2min
[CV 4/5] END ...estimator__C=21.570840306762204;, score=0.835 total time= 4.9min
[CV 5/5] END ...estimator__C=21.570840306762204;, score=0.828 total time= 4.7min
[CV 4/5] END ....estimator__C=49.94310679758585;, score=0.835 total time= 6.4min
[CV 5/5] END ....estimator__C=49.94310679758585

GridSearchCV(cv=MultilabelStratifiedKFold(n_splits=5, random_state=789, shuffle=True),
             estimator=MultiOutputClassifier(estimator=LogisticRegression(max_iter=3000,
                                                                          random_state=789,
                                                                          solver='liblinear'),
                                             n_jobs=-1),
             n_jobs=-1,
             param_grid={'estimator__C': array([21.20229352, 21.57084031, 49.9431068 , 18.14403902, 39.86597279,
        7.75111168, 26.44166646,  7.5585936 , 42.4702...
       24.06662201,  1.51453735, 24.95309398, 25.91529142, 33.06423368,
       18.3129094 , 32.65697313, 48.5215014 ,  2.94057854,  9.62513451,
       27.93192624, 46.24632814, 28.91646816, 15.26946026,  7.67827921,
       23.55246693,  3.72052192, 49.32295268,  7.14234065,  7.49098186,
       35.45740344, 21.72927576, 49.25604696, 31.06075801, 37.17894641,
       23.01446162, 33.467

In [194]:
save_to_pickle(gscv, './gscv.pkl')

In [195]:
print(gscv.best_score_)
print(gscv.best_params_)

0.8327219612199105
{'estimator__C': 3.7205219159979617}


In [196]:
best_model = gscv.best_estimator_

In [197]:
best_model.fit(oof_pred_proba, target)
stacked_test_pred_proba = pd.DataFrame(data=squeeze_pred_proba(best_model.predict_proba(test_pred_proba)), index=test_pred_proba.index, columns=target.columns)

In [201]:
save_to_pickle(best_model, './best_model.pkl')

In [198]:
stacked_oof_pred_proba = cross_val_predict(multiout_model, X=oof_pred_proba, y=target, cv=cv, n_jobs=-1, method='predict_proba')
stacked_oof_pred_proba = pd.DataFrame(data=squeeze_pred_proba(stacked_oof_pred_proba), index=oof_pred_proba.index, columns=target.columns)

In [199]:
opt_treshold = get_one_opt_treshold(target, stacked_oof_pred_proba)

best_score 0.8497969853874406, best_treshold 0.255


In [86]:
# best_score 0.850005522296976, best_treshold 0.264

In [200]:
stacked_test_pred_labels = pd.DataFrame(np.where(stacked_test_pred_proba.values >= opt_treshold, 1, 0), index=test_pred_proba.index, columns=target.columns)
stacked_test_pred_labels_processed = stacked_test_pred_labels.apply(_process_pred_labels, axis=1).rename('target')
stacked_test_pred_labels_processed.to_csv('../submitions/stacked_lr_one_tresh.csv')

# ERRORS ANALYSIS

In [129]:
(stacked_oof_pred_proba >= opt_treshold).sum()

0    22999
1     3597
2        0
3     1105
4        0
5      221
6      631
7      226
8    32179
dtype: int64

In [95]:
stacked_oof_pred_labels = pd.DataFrame(np.where(stacked_oof_pred_proba.values >= opt_treshold, 1, 0), index=oof_pred_proba.index, columns=target.columns)
train = pd.read_pickle('../data/prepared/train.pkl')

In [147]:
s = pd.Series(['a1', 'b2', 'c3'])
s.str.extract(r'([ab])(\d)')

,0,1
0,a,1
1,b,2
2,NaN,NaN


In [160]:
from collections import Counter

In [161]:
t = train.loc[15096, 'positive']

In [162]:
t

'это тест тест тест тест 3848484848448'

In [165]:
c = Counter(t.split())

In [174]:
Counter(t).most_common(2)

[('т', 9), ('8', 6)]

In [178]:
sum([num for word, num in Counter(t.split()).most_common(2)]) / len(t.split())

0.8333333333333334

In [179]:
train.loc[[15096], 'positive'].str.split().str.len()

review_id
15096    6
Name: positive, dtype: int64

In [157]:
train.loc[mask, 'positive'].str.extract(r'([!*\.,?@$-\_()\/+£&]){3,}').drop_duplicates()

,0
review_id,
478,NaN
10710,!
13437,","
14358,*
15096,8
19927,.
20591,-
34189,)
40283,+


In [132]:
TARGET_COL = '3'
mask = (stacked_oof_pred_labels[TARGET_COL] != target[TARGET_COL]) & (target[TARGET_COL] == 1)
print(mask.sum())
train.loc[mask, 'positive'].tolist()

318


['Не хочу писать это противоречит моим принципам.',
 'Задача организации, в особенности же выбранный нами инновационный путь в значительной степени обуславливает создание направлений прогрессивного развития.',
 'Возможность роста и белая зарплата',
 'Все не для тебя, так что лучше не стоит',
 'Всё всё всё всё всё всё всё всё',
 'Доход, условия труда',
 'Я могу работать как удобно работодателю',
 'шикарная премия!!! график работы стандартный. условия работы соответствуют высокому уровню компании. Тренинги, поездки, розыгрыши призов - это приятный бонус от компании.',
 'Вовремя зп, вовремя зп, вовремя зп',
 'Хороший работодатель.',
 'Социальный пакет. Социальный пакет. Социальный пакет. Социальный пакет.Социальный пакет. Социальный пакет. Социальный пакет. Социальный пакет. Социальный пакет. Социальный пакет.',
 'Заработная плата не индексируется по итогам года, в связи с инфляцией. Заработная плата не соответствует должностным обязанностям, почти на всех уровнях рабочих, специалистов и 